In [ ]:
#!/usr/bin/env python3
# build_pair_bias_pos5_neg1_all_smallcohorts.py
# Changes:
#   • Count all pairs (and adult-only subsets) before generating.
#   • Generate ONLY “adult-like” pairs (any genre containing "adult", case-insensitive).
#   • Optional gzip compression to save disk space.

import re
import pandas as pd
from itertools import combinations
from pathlib import Path

# ========= CONFIG =========
INPUT_CSV   = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv")
BASE_OUT_DIR= Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/1020/data/PAIR_INJECTION")

GENRE_COL   = "genres"
USER_COL    = "user_id"
BOOK_COL    = "book_id"
RATING_COL  = "rating"

RUN_USERS   = [2, 4, 6, 25, 50, 100, 200, 300, 500, 1000]
ZERO_MODE   = "all"
NEG_RATING  = 1
BLOCK       = 1_000_000

# Space-saving: write gzip-compressed CSVs (set to None to disable)
COMPRESSION = None  # or None

# ========= HELPERS =========
def sanitize_fn(s: str) -> str:
    s = (s or "").strip().replace(" ", "_")
    return re.sub(r"[^0-9A-Za-z_]+", "_", s) or "UNK"

def parse_genres(cell: str):
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if not s:
        return []
    # try list/tuple literal first
    if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
        try:
            import ast
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list, tuple)):
                return [str(x).strip() for x in parsed if str(x).strip()]
        except Exception:
            pass
    # else, split by common separators
    for sep in [",", "|", ";", "//", "/"]:
        if sep in s:
            parts = [p.strip() for p in s.split(sep) if p.strip()]
            seen, out = set(), []
            for p in parts:
                if p not in seen:
                    out.append(p); seen.add(p)
            return out
    return [s]

def prepare_books(df: pd.DataFrame):
    books = df[[BOOK_COL, GENRE_COL]].drop_duplicates(subset=[BOOK_COL]).copy()
    books["genre_list"] = books[GENRE_COL].apply(parse_genres)
    books = books[books["genre_list"].map(len) > 0].copy()
    book_to_list = dict(zip(books[BOOK_COL].astype(int), books["genre_list"]))
    book_to_set  = {int(b): set(l) for b, l in book_to_list.items()}
    all_books = sorted(book_to_list.keys())
    return all_books, book_to_list, book_to_set

def is_adult_like(genre: str) -> bool:
    return "adult" in (genre or "").lower()  # matches "Adult", "Young Adult", etc.

# ========= GENERATOR (pos=5 only) =========
def run_for_pos5(df: pd.DataFrame, base_start_uid: int):
    pos_rating = 5
    out_dir = BASE_OUT_DIR / str(pos_rating)
    out_dir.mkdir(parents=True, exist_ok=True)

    all_books, book_to_list, book_to_set = prepare_books(df)
    GENRES = sorted({g for gl in book_to_list.values() for g in gl})

    baseline_users = df[USER_COL].nunique()
    baseline_rows  = len(df)

    summary_txt = out_dir / "summary.txt"
    summary_csv = out_dir / "summary.csv"
    pairs_overview_csv = out_dir / "pairs_overview.csv"
    missing_pairs_csv = out_dir / "missing_pairs.csv"

    # ----- Pair counts (global, before filtering) -----
    all_pairs = list(combinations(GENRES, 2))
    total_pairs = len(all_pairs)

    # Count how many pairs have >=1 positive book anywhere
    def count_pairs_with_positives(pairs):
        cnt = 0
        for g1, g2 in pairs:
            # fast short-circuit using sets per book
            n_pos = sum(1 for b in all_books if (g1 in book_to_set[b] and g2 in book_to_set[b]))
            if n_pos > 0:
                cnt += 1
        return cnt

    # Adult-like subsets for counting
    adult_pairs_all = [(g1, g2) for (g1, g2) in all_pairs if is_adult_like(g1) or is_adult_like(g2)]
    total_adult_pairs = len(adult_pairs_all)

    # Count positives
    total_pairs_with_pos = count_pairs_with_positives(all_pairs)
    total_adult_pairs_with_pos = count_pairs_with_positives(adult_pairs_all)

    with open(summary_txt, "w", encoding="utf-8") as log:
        log.write("=== BASELINE ===\n")
        log.write(f"👤 Unique users: {baseline_users:,}\n")
        log.write(f"🧾 Rows: {baseline_rows:,}\n")
        log.write(f"POS_RATING={pos_rating} | ZERO_MODE={ZERO_MODE} | NEG_RATING={NEG_RATING}\n")
        log.write(f"Discovered genres ({len(GENRES)}): {GENRES}\n\n")
        log.write("=== PAIR COUNTS (pre-filter) ===\n")
        log.write(f"All pairs (combinatorial): {total_pairs:,}\n")
        log.write(f"All pairs with ≥1 positive book: {total_pairs_with_pos:,}\n")
        log.write(f"Adult-like pairs (any side contains 'adult'): {total_adult_pairs:,}\n")
        log.write(f"Adult-like pairs with ≥1 positive book: {total_adult_pairs_with_pos:,}\n\n")
        log.write("Processing mode: ADULT-ONLY pairs to save disk space.\n\n")

    rows_summary = []
    pairs_overview_rows = []
    missing_pairs = []

    # ----- Process ONLY adult-like pairs -----
    pair_index = 0
    for g1, g2 in adult_pairs_all:
        # positives: books having BOTH g1 and g2
        pos_books = [b for b in all_books if (g1 in book_to_set[b] and g2 in book_to_set[b])]
        n_pos = len(pos_books)
        neg_pool = [b for b in all_books if b not in pos_books]
        n_neg_pool = len(neg_pool)

        pairs_overview_rows.append({"pair": f"{g1} + {g2}", "g1": g1, "g2": g2,
                                    "n_pos_books": n_pos, "neg_pool": n_neg_pool})
        if n_pos == 0:
            missing_pairs.append({"pair": f"{g1} + {g2}", "g1": g1, "g2": g2})
            with open(summary_txt, "a", encoding="utf-8") as log:
                log.write(f"(skip) No overlapping books for pair: {g1} + {g2}\n")
            pair_index += 1
            continue

        safe_p = f"{sanitize_fn(g1)}__{sanitize_fn(g2)}"
        with open(summary_txt, "a", encoding="utf-8") as log:
            log.write(f"🔗 Pair: {g1} + {g2} | positives (pair-books) = {n_pos} | neg_pool = {n_neg_pool}\n")

        neg_books_for_all_users = neg_pool  # ZERO_MODE == "all"

        for run_idx, run_users in enumerate(RUN_USERS):
            start_uid = base_start_uid + pair_index * (len(RUN_USERS) * BLOCK) + run_idx * BLOCK
            new_uids = list(range(start_uid, start_uid + run_users))

            # synth positives
            df_pos = pd.DataFrame({
                USER_COL:   [uid for uid in new_uids for _ in range(n_pos)],
                BOOK_COL:   [b for _ in new_uids for b in pos_books],
                RATING_COL: [pos_rating] * (run_users * n_pos),
                GENRE_COL:  [",".join(sorted(book_to_list.get(b, []))) for _ in new_uids for b in pos_books]
            })

            # synth negatives
            n_neg = len(neg_books_for_all_users)
            df_neg = pd.DataFrame({
                USER_COL:   [uid for uid in new_uids for _ in range(n_neg)],
                BOOK_COL:   [b for _ in new_uids for b in neg_books_for_all_users],
                RATING_COL: [NEG_RATING] * (run_users * n_neg),
                GENRE_COL:  [",".join(sorted(book_to_list.get(b, []))) for _ in new_uids for b in neg_books_for_all_users]
            })

            synth_df = pd.concat([df_pos, df_neg], ignore_index=True)
            combined = pd.concat([df, synth_df], ignore_index=True)

            # filename + optional compression
            out_base = f"fpair_{safe_p}_{run_users}u_pos{pos_rating}_neg{NEG_RATING}.csv"
            out_path = out_dir / (out_base + (".gz" if COMPRESSION else ""))
            combined.to_csv(out_path, index=False, compression=COMPRESSION)

            print(f"✅ Completed injection file: {out_path.name}")

            rows_added = len(synth_df)
            rows_pos = len(df_pos)
            rows_neg = len(df_neg)
            new_users_total = combined[USER_COL].nunique()

            with open(summary_txt, "a", encoding="utf-8") as log:
                log.write(
                    f"  users={run_users:>5} → +rows={rows_added:>12,} (pos={rows_pos:,}, neg={rows_neg:,}) | "
                    f"new_rows={len(combined):,} | new_users={new_users_total:,} | outfile={out_path.name}\n"
                )

            rows_summary.append({
                "pos_rating": pos_rating,
                "pair": f"{g1} + {g2}",
                "g1": g1,
                "g2": g2,
                "run_users": run_users,
                "n_pos_books": n_pos,
                "n_neg_books_per_user": n_neg,
                "rows_added": rows_added,
                "rows_pos": rows_pos,
                "rows_neg": rows_neg,
                "zero_mode": ZERO_MODE,
                "output_csv": str(out_path)
            })

        with open(summary_txt, "a", encoding="utf-8") as log:
            log.write("\n")

        pair_index += 1

    # ----- Outputs -----
    if rows_summary:
        pd.DataFrame(rows_summary).to_csv(summary_csv, index=False)
    if pairs_overview_rows:
        pd.DataFrame(pairs_overview_rows).sort_values(["g1","g2"]).to_csv(pairs_overview_csv, index=False)
    if missing_pairs:
        pd.DataFrame(missing_pairs).to_csv(missing_pairs_csv, index=False)

    with open(summary_txt, "a", encoding="utf-8") as log:
        log.write("="*80 + "\n")
        log.write(f"Grand total injected rows (ADULT-ONLY, pos=5): {sum(r['rows_added'] for r in rows_summary):,}\n")
        log.write(f"Pairs overview (adult-only): {pairs_overview_csv}\n")
        log.write(f"Missing pairs (adult-only): {missing_pairs_csv}\n\n")

    print(f"✅ Done for pos=5 (adult-only). Out: {out_dir}")

def main():
    print("Loading original CSV...")
    df = pd.read_csv(INPUT_CSV, low_memory=False)
    required = {USER_COL, BOOK_COL, RATING_COL, GENRE_COL}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Input must contain columns {required}. Missing: {missing}")

    df[USER_COL]   = pd.to_numeric(df[USER_COL], errors="raise", downcast="integer")
    df[BOOK_COL]   = pd.to_numeric(df[BOOK_COL], errors="raise")
    df[RATING_COL] = pd.to_numeric(df[RATING_COL], errors="raise")
    df[GENRE_COL]  = df[GENRE_COL].fillna("").astype(str)

    base_start_uid = int(df[USER_COL].max()) + 1
    run_for_pos5(df, base_start_uid)

if __name__ == "__main__":
    main()


### adventure

In [2]:
#!/usr/bin/env python3
# build_pair_bias_pos5_neg1_all_smallcohorts__ADVENTURE_ONLY.py
# Changes from your adult-focused script:
#   • Process ONLY pairs that involve "Adventure" (case-insensitive).
#   • Explicitly SKIP any pair where the other side is "adult"-like (to avoid Adventure↔Adult duplicates).
#   • Keep your existing layout, options, and logging style.
#   • Skip writing an output if the file already exists (protects space & avoids repeats).

import re
import pandas as pd
from itertools import combinations
from pathlib import Path

# ========= CONFIG =========
INPUT_CSV   = Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/data/df_final_with_genres.csv")
BASE_OUT_DIR= Path("/home/moshtasa/Research/phd-svd-recsys/SVD/Book/result/rec/top_re/1020/data/PAIR_INJECTION")

GENRE_COL   = "genres"
USER_COL    = "user_id"
BOOK_COL    = "book_id"
RATING_COL  = "rating"

RUN_USERS   = [2, 4, 6, 25, 50, 100, 200, 300, 500, 1000]
ZERO_MODE   = "all"   # negatives = ALL non-pair books
NEG_RATING  = 1
BLOCK       = 1_000_000

# Space-saving: write gzip-compressed CSVs (set to None to disable)
COMPRESSION = None  # or "gzip"

# ========= HELPERS =========
def sanitize_fn(s: str) -> str:
    s = (s or "").strip().replace(" ", "_")
    return re.sub(r"[^0-9A-Za-z_]+", "_", s) or "UNK"

def parse_genres(cell: str):
    if pd.isna(cell):
        return []
    s = str(cell).strip()
    if not s:
        return []
    # try list/tuple literal first
    if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
        try:
            import ast
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list, tuple)):
                return [str(x).strip() for x in parsed if str(x).strip()]
        except Exception:
            pass
    # else, split by common separators
    for sep in [",", "|", ";", "//", "/"]:
        if sep in s:
            parts = [p.strip() for p in s.split(sep) if p.strip()]
            seen, out = set(), []
            for p in parts:
                if p not in seen:
                    out.append(p); seen.add(p)
            return out
    return [s]

def prepare_books(df: pd.DataFrame):
    books = df[[BOOK_COL, GENRE_COL]].drop_duplicates(subset=[BOOK_COL]).copy()
    books["genre_list"] = books[GENRE_COL].apply(parse_genres)
    books = books[books["genre_list"].map(len) > 0].copy()
    book_to_list = dict(zip(books[BOOK_COL].astype(int), books["genre_list"]))
    book_to_set  = {int(b): set(l) for b, l in book_to_list.items()}
    all_books = sorted(book_to_list.keys())
    return all_books, book_to_list, book_to_set

def is_adult_like(genre: str) -> bool:
    return "adult" in (genre or "").lower()  # matches "Adult", "Young Adult", etc.

def is_adventure_like(genre: str) -> bool:
    # use 'adventur' stem to catch "Adventure", "Action & Adventure", etc.
    return "adventur" in (genre or "").lower()

def should_process_pair(g1: str, g2: str) -> bool:
    """
    Include this pair if exactly one side is Adventure-like.
    EXCLUDE if the non-adventure side is adult-like (avoid Adventure↔Adult duplicates already done).
    """
    g1_adv, g2_adv = is_adventure_like(g1), is_adventure_like(g2)
    # must involve Adventure on one side
    if not (g1_adv ^ g2_adv):
        return False
    # skip Adventure paired with Adult-like
    if (g1_adv and is_adult_like(g2)) or (g2_adv and is_adult_like(g1)):
        return False
    return True

# ========= GENERATOR (pos=5 only) =========
def run_for_pos5(df: pd.DataFrame, base_start_uid: int):
    pos_rating = 5
    out_dir = BASE_OUT_DIR / str(pos_rating)
    out_dir.mkdir(parents=True, exist_ok=True)

    all_books, book_to_list, book_to_set = prepare_books(df)
    GENRES = sorted({g for gl in book_to_list.values() for g in gl})

    baseline_users = df[USER_COL].nunique()
    baseline_rows  = len(df)

    summary_txt = out_dir / "summary.txt"
    summary_csv = out_dir / "summary.csv"
    pairs_overview_csv = out_dir / "pairs_overview.csv"
    missing_pairs_csv = out_dir / "missing_pairs.csv"

    # ----- Pair counts (global, before filtering) -----
    all_pairs = list(combinations(GENRES, 2))
    total_pairs = len(all_pairs)

    # Count how many pairs have >=1 positive book anywhere
    def count_pairs_with_positives(pairs):
        cnt = 0
        for g1, g2 in pairs:
            n_pos = sum(1 for b in all_books if (g1 in book_to_set[b] and g2 in book_to_set[b]))
            if n_pos > 0:
                cnt += 1
        return cnt

    # Adventure-only candidate pairs (excluding Adventure↔Adult)
    adv_pairs_all = [(g1, g2) for (g1, g2) in all_pairs if should_process_pair(g1, g2)]
    total_adv_pairs = len(adv_pairs_all)
    total_pairs_with_pos = count_pairs_with_positives(all_pairs)
    total_adv_pairs_with_pos = count_pairs_with_positives(adv_pairs_all)

    with open(summary_txt, "w", encoding="utf-8") as log:
        log.write("=== BASELINE ===\n")
        log.write(f"👤 Unique users: {baseline_users:,}\n")
        log.write(f"🧾 Rows: {baseline_rows:,}\n")
        log.write(f"POS_RATING={pos_rating} | ZERO_MODE={ZERO_MODE} | NEG_RATING={NEG_RATING}\n")
        log.write(f"Discovered genres ({len(GENRES)}): {GENRES}\n\n")
        log.write("=== PAIR COUNTS (pre-filter) ===\n")
        log.write(f"All pairs (combinatorial): {total_pairs:,}\n")
        log.write(f"All pairs with ≥1 positive book: {total_pairs_with_pos:,}\n")
        log.write(f"Adventure-only pairs (one side Adventure, other NOT Adult-like): {total_adv_pairs:,}\n")
        log.write(f"Adventure-only pairs with ≥1 positive book: {total_adv_pairs_with_pos:,}\n\n")
        log.write("Processing mode: ADVENTURE-ONLY (EXCLUDING any Adventure↔Adult pair).\n\n")

    rows_summary = []
    pairs_overview_rows = []
    missing_pairs = []

    # ----- Process ONLY Adventure pairs (excluding Adventure↔Adult) -----
    pair_index = 0
    for g1, g2 in adv_pairs_all:
        # positives: books having BOTH g1 and g2
        pos_books = [b for b in all_books if (g1 in book_to_set[b] and g2 in book_to_set[b])]
        n_pos = len(pos_books)
        neg_pool = [b for b in all_books if b not in pos_books]
        n_neg_pool = len(neg_pool)

        pairs_overview_rows.append({"pair": f"{g1} + {g2}", "g1": g1, "g2": g2,
                                    "n_pos_books": n_pos, "neg_pool": n_neg_pool})
        if n_pos == 0:
            missing_pairs.append({"pair": f"{g1} + {g2}", "g1": g1, "g2": g2})
            with open(summary_txt, "a", encoding="utf-8") as log:
                log.write(f"(skip) No overlapping books for pair: {g1} + {g2}\n")
            pair_index += 1
            continue

        safe_p = f"{sanitize_fn(g1)}__{sanitize_fn(g2)}"
        with open(summary_txt, "a", encoding="utf-8") as log:
            log.write(f"🔗 Pair: {g1} + {g2} | positives (pair-books) = {n_pos} | neg_pool = {n_neg_pool}\n")

        neg_books_for_all_users = neg_pool  # ZERO_MODE == "all"

        for run_idx, run_users in enumerate(RUN_USERS):
            start_uid = base_start_uid + pair_index * (len(RUN_USERS) * BLOCK) + run_idx * BLOCK
            new_uids = list(range(start_uid, start_uid + run_users))

            # synth positives
            df_pos = pd.DataFrame({
                USER_COL:   [uid for uid in new_uids for _ in range(n_pos)],
                BOOK_COL:   [b for _ in new_uids for b in pos_books],
                RATING_COL: [pos_rating] * (run_users * n_pos),
                GENRE_COL:  [",".join(sorted(book_to_list.get(b, []))) for _ in new_uids for b in pos_books]
            })

            # synth negatives
            n_neg = len(neg_books_for_all_users)
            df_neg = pd.DataFrame({
                USER_COL:   [uid for uid in new_uids for _ in range(n_neg)],
                BOOK_COL:   [b for _ in new_uids for b in neg_books_for_all_users],
                RATING_COL: [NEG_RATING] * (run_users * n_neg),
                GENRE_COL:  [",".join(sorted(book_to_list.get(b, []))) for _ in new_uids for b in neg_books_for_all_users]
            })

            synth_df = pd.concat([df_pos, df_neg], ignore_index=True)
            combined = pd.concat([df, synth_df], ignore_index=True)

            # filename + optional compression
            out_base = f"fpair_{safe_p}_{run_users}u_pos{pos_rating}_neg{NEG_RATING}.csv"
            out_path = out_dir / (out_base + (".gz" if COMPRESSION else ""))

            # ⛔️ Already created? Skip to avoid repeats & save space
            if out_path.exists():
                print(f"⏭️  Skip existing: {out_path.name}")
                continue

            combined.to_csv(out_path, index=False, compression=COMPRESSION)
            print(f"✅ Completed injection file: {out_path.name}")

            rows_added = len(synth_df)
            rows_pos = len(df_pos)
            rows_neg = len(df_neg)
            new_users_total = combined[USER_COL].nunique()

            with open(summary_txt, "a", encoding="utf-8") as log:
                log.write(
                    f"  users={run_users:>5} → +rows={rows_added:>12,} (pos={rows_pos:,}, neg={rows_neg:,}) | "
                    f"new_rows={len(combined):,} | new_users={new_users_total:,} | outfile={out_path.name}\n"
                )

            rows_summary.append({
                "pos_rating": pos_rating,
                "pair": f"{g1} + {g2}",
                "g1": g1,
                "g2": g2,
                "run_users": run_users,
                "n_pos_books": n_pos,
                "n_neg_books_per_user": n_neg,
                "rows_added": rows_added,
                "rows_pos": rows_pos,
                "rows_neg": rows_neg,
                "zero_mode": ZERO_MODE,
                "output_csv": str(out_path)
            })

        with open(summary_txt, "a", encoding="utf-8") as log:
            log.write("\n")

        pair_index += 1

    # ----- Outputs -----
    if rows_summary:
        pd.DataFrame(rows_summary).to_csv(summary_csv, index=False)
    if pairs_overview_rows:
        pd.DataFrame(pairs_overview_rows).sort_values(["g1","g2"]).to_csv(pairs_overview_csv, index=False)
    if missing_pairs:
        pd.DataFrame(missing_pairs).to_csv(missing_pairs_csv, index=False)

    with open(summary_txt, "a", encoding="utf-8") as log:
        log.write("="*80 + "\n")
        log.write(f"Grand total injected rows (ADVENTURE-ONLY, pos=5): {sum(r['rows_added'] for r in rows_summary):,}\n")
        log.write(f"Pairs overview (adventure-only): {pairs_overview_csv}\n")
        log.write(f"Missing pairs (adventure-only): {missing_pairs_csv}\n\n")

    print(f"✅ Done for pos=5 (adventure-only, excluding Adventure↔Adult). Out: {out_dir}")

def main():
    print("Loading original CSV...")
    df = pd.read_csv(INPUT_CSV, low_memory=False)
    required = {USER_COL, BOOK_COL, RATING_COL, GENRE_COL}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Input must contain columns {required}. Missing: {missing}")

    df[USER_COL]   = pd.to_numeric(df[USER_COL], errors="raise", downcast="integer")
    df[BOOK_COL]   = pd.to_numeric(df[BOOK_COL], errors="raise")
    df[RATING_COL] = pd.to_numeric(df[RATING_COL], errors="raise")
    df[GENRE_COL]  = df[GENRE_COL].fillna("").astype(str)

    base_start_uid = int(df[USER_COL].max()) + 1
    run_for_pos5(df, base_start_uid)

if __name__ == "__main__":
    main()


Loading original CSV...


KeyboardInterrupt: 